In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("veriler.csv")
df.head()

,ulke,boy,kilo,yas,cinsiyet
0,tr,130,30,10,e
1,tr,125,36,11,e
2,tr,135,34,10,k
3,tr,133,30,9,k
4,tr,129,38,12,e


In [3]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

In [4]:
le = LabelEncoder()
ohe = OneHotEncoder()

In [5]:
df["cinsiyet"] = le.fit_transform(df["cinsiyet"])

In [6]:
df.head()

,ulke,boy,kilo,yas,cinsiyet
0,tr,130,30,10,0
1,tr,125,36,11,0
2,tr,135,34,10,1
3,tr,133,30,9,1
4,tr,129,38,12,0


In [13]:
ulkeDummie = pd.get_dummies(df["ulke"])

,fr,tr,us
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,0,1,0
6,0,1,0
7,0,1,0
8,0,1,0
9,0,0,1


In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop("cinsiyet" , axis = 1)
y = df["cinsiyet"]

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size = 0.3 , random_state = 53)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train , y_train)